In [1]:
import os
import pickle

import pandas as pd

In [2]:
!pip install --upgrade keras

import keras

print(keras.__version__)

3.9.1


In [3]:
# Define the project's root directory (replace with your actual path)
project_root = "/Users/sunnythesage/PythonProjects/Data-Science-BootCamp/03-Deep-Learning-BootCamp/7 - End to End Deep Learning Project Using ANN/advanced-customer-churn-analysis-using-ann"

# Change the current working directory to the project's root
os.chdir(project_root)

# Define the artifacts directory path
artifacts_dir = os.path.join(os.getcwd(), 'artifacts')

# Load the trained model
model_path = os.path.join(artifacts_dir, 'model.keras')  # use .keras
model = keras.saving.load_model(model_path)

# Load the encoder and scaler
with open(os.path.join(artifacts_dir, 'onehot_encoder_geo.pkl'), 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open(os.path.join(artifacts_dir, 'label_encoder_gender.pkl'), 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open(os.path.join(artifacts_dir, 'scaler.pkl'), 'rb') as file:
    scaler = pickle.load(file)

In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
# One-hot encode 'Geography'
# Create a DataFrame with the 'Geography' value
geo_df = pd.DataFrame([input_data['Geography']], columns = ['Geography'])

# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform(geo_df).toarray()

# Create a DataFrame from the encoded data
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot_encoder_geo.get_feature_names_out(['Geography']))

geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
## Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
## concatenation one hot encoded
input_df = pd.concat([input_df.drop("Geography", axis = 1), geo_encoded_df], axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [10]:
## PRedict churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[0.0141239]], dtype=float32)

In [11]:
prediction_proba = prediction[0][0]

In [12]:
prediction_proba

0.014123895

In [13]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
